For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

$$\begin{aligned}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
\end{aligned}$$

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\text{IRR}(K)} \\
      h'(K) &= \frac{\text{IRR}(K)g'(K) - g(K)\text{IRR}'(K)}{\text{IRR}(K)^2} \\
      h''(K) &= \frac{\text{IRR}(K)g''(K)-\text{IRR}''(K)g(K) -2\cdot\text{IRR}'(K)g'(K)}{\text{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\text{IRR}'(K)^2g(K)}{\text{IRR}(K)^3}.
    \end{split}
  \end{equation*}
  
For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

Let $m$ denote the payment frequency ($m=2$ for semi-annual payment frequency), and let $N = T_N-T_n$ denote the tenor of the swap (number of years), the partial derivatives on the IRR function $\text{IRR}(S)$ given by:
\begin{equation*}
\begin{split}
\text{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\text{IRR}'(K)&=-\frac{1}{K}\text{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\text{IRR}''(K)&=-\frac{2}{K}\text{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

These results will need to be generalised to handle the case for $m=2$ to be consistent with the semi-annual payment frequency swap market data provided.

---
For CMS rate payment, since $g(F)=F$, we have the derivatives:

\begin{equation*}
\begin{split}
g(K) &= K \\
g'(K) &= 1 \\
g''(K) &= 0
\end{split}
\end{equation*}

# Imports and data wrangling

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq
from scipy.integrate import quad
from scipy.stats import norm

ois = pd.read_csv('./data/ois.csv')     # load discounting curve
fsr = pd.read_csv('./data/sabr_params.csv')     # load discounting curve

In [ ]:
## IRR funcions
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative
# Payoff functions
def g_0(K):
    return K

def g_1(K):
    return 1.0

def g_2(K):
    return 0.0

def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

# def pvbp(expiry, tenor):
#     # Calculate PVBP
#     """
#     Return PVBP for a period from expiry to end of tenor in years t should be >= 0.5
#     assumed to be semi-annual payments
#     """
#     t = expiry
#     T = expiry + tenor
#     delta_t = 0.5   
#     sum_df = (ois.loc[(ois.years > t) & (ois.years <= T)]).df.sum()

#     return delta_t * sum_df
    
def black76_pay(F, K, expiry, tenor, sigma):
    """
    Return value of payer swaption via Black Normal model only
    parameters
        F: forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * (sigma**2) * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = F * norm.cdf(d_1) - K * norm.cdf(d_2)
    return black_option
    
def black76_rec(F, K, expiry, tenor, sigma):
    """
    Return value of receiver swaption via Black Normal model Only
    parameters
        F: Forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * (sigma**2) * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = K * norm.cdf(-d_2) - F * norm.cdf(-d_1)
    return black_option

def SABR(F, K, T, alpha, rho, nu, beta=0.9):
    """
    Use SABR model to return sigma, set beta to be 0.9
    """
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [ ]:
def rec_integrand(F, K, expiry, tenor, alpha, rho, nu, m):
    """
    Return value of receiver integrand per CMS rate payoff. 
    formula: 2nd derivative h * V_rec(per SABR valuation) 
    parameters:
        m: payment freq per year, e.g. 2 is semi-annual
        F: par swap rate
        K: strike of CMS
        expiry: years to expiry
        alpha: SABR param
        rho: SABR param
        nu: SABR param
    """
    sigma_sabr = SABR(F, K, expiry, alpha, rho, nu, beta=0.9)
    sabr_pricing = black76_rec(F, K, expiry, tenor, sigma_sabr)
    return h_2(K, m, tenor) * sabr_pricing

def pay_integrand(F, K, expiry, tenor, alpha, rho, nu, m):
    """
    Return value of receiver integrand per CMS rate payoff. 
    formula: 2nd derivative h * V_rec(per SABR valuation) 
    parameters:
        m: payment freq per year, e.g. 2 is semi-annual
        F: par swap rate
        K: strike of CMS
        expiry: years to expiry
        alpha: SABR param
        rho: SABR param
        nu: SABR param
    """
    sigma_sabr = SABR(F, K, expiry, alpha, rho, nu, beta=0.9)
    sabr_pricing = black76_pay(F, K, expiry, tenor, sigma_sabr)
    return h_2(K, m, tenor) * sabr_pricing

def v_0(F, expiry, tenor, alpha, rho, nu, m):
    """
    Return CMS pay off based of static replication discounted to today's price. Uses SABR model
    
    """
    df_expiry = ois.loc[ois.years == expiry]['df'].values[0]
    int_1 = IRR_0(F) * quad(lambda x: rec_integrand(F, x, expiry, tenor, alpha, rho, nu, m), 0.0, F)
    int_2 = IRR_0(F) * quad(lambda x: pay_integrand(F, x, expiry, tenor, alpha, rho, nu, m), F, 0.1)

    return df_expiry * F + int_1 + int_2



In [ ]:



I_put = quad(lambda x: putintegrand(x, S, r, T, sigma), 0.0, F)
I_call = quad(lambda x: callintegrand(x, S, r, T, sigma), F, 5000)
E_var = 2*np.exp(r*T)*(I_put[0] + I_call[0])
print('The expected integrated variance is: %.9f' % E_var)

In [ ]:
dd